In [1]:
import pandas as pd
import os
from collections import OrderedDict
import math





def findRange(thresholds, v):
    for i, th in enumerate(thresholds):
        if(v <= th):
            if i==0:
                return "x<{}".format(th)
            elif i == len(thresholds)-1:
                return "x>{}".format(thresholds[i-1])
            else:
                return "{}<x<{}".format(thresholds[i-1], thresholds[i])

def convert_cate(arr):
    n = 4 #parts to be divided
    maxValue = max(arr)
    minValue = min(arr)
    thresholds = [ math.floor(i*(maxValue-minValue)/n)+minValue for i in range(n+1)]

    print([findRange(thresholds, i) for i in arr])
    
    return pd.Series([findRange(thresholds, i) for i in arr])


def num2cate(df, num2cate_key, remove_key):
#     new_data = pd.DataFrame()
    for k in df.columns:
        print(len(k))
        if (k in remove_key):
            print('remove key', k)
            df.drop(columns=[k])
            continue 
        elif k in num2cate_key:
            print('modify', k)
            values = pd.to_numeric(df[k])
            df[k] = convert_cate(values.tolist())

        else:
            print('same key', k)

    for k in df:
        print(k)
        
    return df


def split(df):
    sex, marriage = df['personal_status'].str.split(' ', 1).str
    df.insert(loc=0, column='sex', value=sex)
    df.insert(loc=0, column='marriage', value=marriage)
    df.drop('personal_status', axis = 1, inplace=True)
    
    return df

In [3]:
filename = "credit.data.csv"
data_dir = os.path.join('../data/credit',filename)
df = pd.read_csv(data_dir)

# print(df.columns)

# num2cate_key = ["credit_amount", "duration", "age"]
# remove_key = ["education_num"]

# # num to cate
# df = num2cate(df, num2cate_key, remove_key)

#
df = split(df)



df.to_csv("credit_clean.csv", index=False)

In [7]:
filename = "adult.test.csv"
data_dir = os.path.join('../data/adult',filename)
df = pd.read_csv(data_dir)

df['country']= df['country'].apply(lambda x: 'US' if x == 'United-States' else 'non-US')

df.to_csv("../data/adult/adult.test.csv", index=False)

In [20]:
import os 
import pandas as pd

def json2csv(data, model):
    filename = "{}_{}_rules.json".format(data, model)
    data_dir = os.path.join('../../front/src/asset',filename)
    df = pd.read_json(data_dir)
    df=df.drop(columns=['elift', 'id','sup_pd','sup_pnd'])
    
    df['size']= df.apply(lambda row: len(row['items']), axis = 1) 
    
    if data=='adult':
        df = df[abs(df['risk_dif'])>=0.1]
    elif data=='academic':
        df = df[abs(df['risk_dif'])>=0.25]
    
    save_path='../../../discrilens/'
    df.to_csv(save_path+filename.replace('.json','.csv'), index=False)

In [21]:
datasets=['academic', 'adult']
models = ['knn','rf','dt','svm','lr','xgb']
for data in datasets:
    for m in models:
        json2csv(data, m)

In [18]:
abs(-0.1)

0.1